In [185]:
import pandas as pd

df = pd.read_excel("WaitTime.xlsx", sheet_name="Wait times 2008 to 2023", header=2)
df.head()

,Reporting level,Province/territory,Region,Indicator,Metric,Data year,Unit of measurement,Indicator result,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Provincial,Alberta,NaN,Bladder Cancer Surgery,50th Percentile,2008,Days,NaN,NaN,NaN,NaN,NaN
1,Provincial,Alberta,NaN,Bladder Cancer Surgery,90th Percentile,2008,Days,NaN,NaN,NaN,NaN,NaN
2,Provincial,Alberta,NaN,Bladder Cancer Surgery,Volume,2008,Number of cases,NaN,NaN,NaN,NaN,NaN
3,Provincial,Alberta,NaN,Breast Cancer Surgery,50th Percentile,2008,Days,NaN,NaN,NaN,NaN,NaN
4,Provincial,Alberta,NaN,Breast Cancer Surgery,90th Percentile,2008,Days,NaN,NaN,NaN,NaN,NaN


In [212]:
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df = df.drop(columns=["Region"], errors="ignore")

df.head()

,Reporting level,Province/territory,Indicator,Metric,Data year,Unit of measurement,Indicator result
0,Provincial,Alberta,Bladder Cancer Surgery,50th Percentile,2008.0,Days,NaN
1,Provincial,Alberta,Bladder Cancer Surgery,90th Percentile,2008.0,Days,NaN
2,Provincial,Alberta,Bladder Cancer Surgery,Volume,2008.0,Number of cases,NaN
3,Provincial,Alberta,Breast Cancer Surgery,50th Percentile,2008.0,Days,NaN
4,Provincial,Alberta,Breast Cancer Surgery,90th Percentile,2008.0,Days,NaN


In [188]:
def standardize_unit_of_measurement(row):
    if row["Metric"] == "50th Percentile" and str( row["Unit of measurement"]).lower() == "days":
        return row["Indicator result"] * 24
    return row["Indicator result"]

def get_unit_of_measurement(row):
    if row["Metric"] == "50th Percentile" and str( row["Unit of measurement"]).lower() == "days":
        return "Hours"
    return row["Unit of measurement"]

def calculate_missing_summary(df, group_by_col="Metric", value_col="Indicator result"):
    total_counts = df.groupby(group_by_col)[value_col].count() + df.groupby(
        group_by_col
    )[value_col].apply(lambda x: x.isna().sum())

    null_counts = df.groupby(group_by_col)[value_col].apply(lambda x: x.isna().sum())

    missing_percentage = (null_counts / total_counts * 100).round(2)

    missing_summary = pd.DataFrame(
        {"Total": total_counts, "Missing": null_counts, "Missing %": missing_percentage}
    )

    return missing_summary

In [213]:
df["Data year"] = pd.to_numeric(df["Data year"], errors="coerce")
df_filtered = df[(df["Data year"] >= 2013) & (df["Data year"] <= 2023)]

df_filtered = df_filtered[df_filtered["Reporting level"] == "Provincial"]

df_filtered["Indicator result"] = df_filtered.apply(
    standardize_unit_of_measurement, axis=1
)
df_filtered["Unit of measurement"] = df_filtered.apply(get_unit_of_measurement, axis=1)

df_filtered = df_filtered.reset_index(drop=True)
df_filtered["Data year"] = df_filtered["Data year"].astype(int)
df_filtered.head()

,Reporting level,Province/territory,Indicator,Metric,Data year,Unit of measurement,Indicator result
0,Provincial,Alberta,Bladder Cancer Surgery,50th Percentile,2013,Hours,672.0
1,Provincial,Alberta,Bladder Cancer Surgery,90th Percentile,2013,Days,57.0
2,Provincial,Alberta,Bladder Cancer Surgery,Volume,2013,Number of cases,477.0
3,Provincial,Alberta,Breast Cancer Surgery,50th Percentile,2013,Hours,360.0
4,Provincial,Alberta,Breast Cancer Surgery,90th Percentile,2013,Days,40.0


In [214]:
df_filtered = df_filtered[df_filtered["Metric"] != "90th Percentile"] 
df_filtered = df_filtered[df_filtered["Metric"] != "% Meeting Benchmark"]
df_filtered.head()


,Reporting level,Province/territory,Indicator,Metric,Data year,Unit of measurement,Indicator result
0,Provincial,Alberta,Bladder Cancer Surgery,50th Percentile,2013,Hours,672.0
2,Provincial,Alberta,Bladder Cancer Surgery,Volume,2013,Number of cases,477.0
3,Provincial,Alberta,Breast Cancer Surgery,50th Percentile,2013,Hours,360.0
5,Provincial,Alberta,Breast Cancer Surgery,Volume,2013,Number of cases,1169.0
6,Provincial,Alberta,CABG,50th Percentile,2013,Hours,288.0


In [215]:
missing_summary = calculate_missing_summary(df_filtered)
print(missing_summary)


                 Total  Missing  Missing %
Metric                                    
50th Percentile   1540      251      16.30
Volume            1540      208      13.51


In [216]:
df_filtered["Indicator result"] = df_filtered.groupby(
    ["Province/territory", "Indicator", "Metric"]
)["Indicator result"].transform(lambda x: x.fillna(x.median()))

volume_all_null_groups = (
    df_filtered[df_filtered["Metric"] == "Volume"]
    .groupby(["Province/territory", "Indicator", "Metric"])["Indicator result"]
    .apply(lambda x: x.isna().all())
)

volume_fill_groups = volume_all_null_groups[volume_all_null_groups].index

df_filtered["Indicator result"] = df_filtered.apply(
    lambda row: 0
    if (
        row["Metric"] == "Volume"
        and (row["Province/territory"], row["Indicator"], row["Metric"])
        in volume_fill_groups
    )
    else row["Indicator result"],
    axis=1,
)


df_filtered["Indicator result"] = df_filtered.groupby(["Indicator", "Metric"])[
    "Indicator result"
].transform(lambda x: x.fillna(x.median()))


unfilled_50th = df_filtered[
    (df_filtered["Metric"] == "50th Percentile")
    & df_filtered["Indicator result"].isna()
][["Province/territory", "Indicator"]].drop_duplicates()


drop_keys = set([tuple(x) for x in unfilled_50th.values])

df_filtered = df_filtered[
    ~df_filtered[["Province/territory", "Indicator"]]
    .apply(tuple, axis=1)
    .isin(drop_keys)
].reset_index(drop=True)

missing_summary = calculate_missing_summary(df_filtered)
print(missing_summary)

                 Total  Missing  Missing %
Metric                                    
50th Percentile   1540        0        0.0
Volume            1540        0        0.0


In [217]:

df_filtered.to_csv("./CleanedWaitTime.csv", index=False)